In [82]:
# import numpy as np
import pickle

class Agent():
    
    def __init__(self,move_value):
        self.move_value = move_value
        self.value = dict()
    
    def state(self,current_state):
        if str(current_state) in self.value:
            pass
        else: 
            self.value[str(current_state)] = 0

    def action(self):
        pass
    def reward(self):
        pass

class Environment():
    
    def __init__(self):
        self.board = np.zeros((3,3))
    
    # Def: Check if there is any winner
    def check_winner(self):
        
        # ======== Check Winner By One Row ========         
        for row in self.board:
            if sum(row) == 3: return 1
            if sum(row) ==-3: return -1 
            
        for row in np.transpose(self.board):
            if sum(row) == 3: return 1
            if sum(row) ==-3: return -1 
        # ======== Check Winner By One Row ========         

                    
        # ======== Check Winner By Diag ========         
        if sum([self.board[i][i] for i in range(len(self.board))]) == 3:  return 1
        if sum([self.board[i][i] for i in range(len(self.board))]) == -3: return -1
        if sum([self.board[i][len(self.board)-i-1] for i in range(len(self.board))]) == 3: return  1       
        if sum([self.board[i][len(self.board)-i-1] for i in range(len(self.board))]) == -3:return -1  
        # ======== Check Winner By Diag ========         

        # ======== Check Draw ========         
        if abs(self.board).sum() == 9: return 0
        # ======== Check Draw ========         
            
    # Def: Check if there is a winner
    def check_if_there_is_winner(self): return self.check_winner()
    
    # change the borad position - location and value.      
    def make_a_move(self,position,value):
        assert (value == 1 or value == -1)
        assert (self.board.ravel()[position] == 0)
        self.board.ravel()[position] = value
    
    # Reset the board back the starting position.     
    def reset_game(self): self.board = np.zeros((3,3))
        
    # View the current state of the board
    def get_board(self): return self.board

In [96]:
temp = Environment()
temp.make_a_move(0,-1)
temp.make_a_move(1,1)
temp.make_a_move(8,1)
print(temp.get_board())
print(temp.check_if_there_is_winner())
temp.make_a_move(2,-1)
temp.make_a_move(4,1)
temp.make_a_move(5,-1)
temp.make_a_move(3,1)
temp.make_a_move(7,-1)
temp.make_a_move(6,-1)
print(temp.get_board())
print(temp.check_if_there_is_winner())


[[-1.  1.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  1.]]
None
[[-1.  1. -1.]
 [ 1.  1. -1.]
 [-1. -1.  1.]]
0


In [62]:
temp_dict = dict()
temp_dict[str(temp.get_board())] = 4
temp_dict

{'[[ 1.  1.  1.]\n [ 0.  0.  0.]\n [ 0.  0. -1.]]': 4}

In [71]:
temp.make_a_move(4,1)
temp.make_a_move(4,1)

AssertionError: 

In [80]:
abs(temp.get_board()).sum()

5.0

In [97]:
! git all-go

[master 475cfa7] commit
 2 files changed, 916 insertions(+), 2900 deletions(-)
 rewrite .ipynb_checkpoints/b reference -checkpoint.ipynb (82%)
 rewrite b reference .ipynb (82%)
Counting objects: 5, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 1.10 KiB | 1.10 MiB/s, done.
Total 5 (delta 4), reused 0 (delta 0)
remote: Resolving deltas: 100% (4/4), completed with 3 local objects.
To https://github.com/JaeDukSeo/tictactoe.git
   3ec61b9..475cfa7  master -> master


In [101]:
import numpy as np

BOARD_ROWS = 3
BOARD_COLS = 3
BOARD_SIZE = BOARD_ROWS * BOARD_COLS

class State:
    
    def __init__(self):
        # the board is represented by an n * n array,
        # 1 represents a chessman of the player who moves first,
        # -1 represents a chessman of another player
        # 0 represents an empty position
        self.data = np.zeros((BOARD_ROWS, BOARD_COLS))
        self.winner = None
        self.hash_val = None
        self.end = None

    # compute the hash value for one state, it's unique
    def hash(self):
        if self.hash_val is None:
            self.hash_val = 0
            for i in self.data.reshape(BOARD_ROWS * BOARD_COLS):
                if i == -1:
                    i = 2
                self.hash_val = self.hash_val * 3 + i
        return int(self.hash_val)

    # check whether a player has won the game, or it's a tie
    def is_end(self):
        if self.end is not None:
            return self.end
        results = []
        # check row
        for i in range(0, BOARD_ROWS):
            results.append(np.sum(self.data[i, :]))
        # check columns
        for i in range(0, BOARD_COLS):
            results.append(np.sum(self.data[:, i]))

        # check diagonals
        results.append(0)
        for i in range(0, BOARD_ROWS):
            results[-1] += self.data[i, i]
        results.append(0)
        for i in range(0, BOARD_ROWS):
            results[-1] += self.data[i, BOARD_ROWS - 1 - i]

        for result in results:
            if result == 3:
                self.winner = 1
                self.end = True
                return self.end
            if result == -3:
                self.winner = -1
                self.end = True
                return self.end

        # whether it's a tie
        sum = np.sum(np.abs(self.data))
        if sum == BOARD_ROWS * BOARD_COLS:
            self.winner = 0
            self.end = True
            return self.end

        # game is still going on
        self.end = False
        return self.end

    # @symbol: 1 or -1
    # put chessman symbol in position (i, j)
    def next_state(self, i, j, symbol):
        new_state = State()
        new_state.data = np.copy(self.data)
        new_state.data[i, j] = symbol
        return new_state
    
    # print the board
    def print_state(self):
        for i in range(0, BOARD_ROWS):
            print('-------------')
            out = '| '
            for j in range(0, BOARD_COLS):
                if self.data[i, j] == 1:
                    token = '*'
                if self.data[i, j] == 0:
                    token = '0'
                if self.data[i, j] == -1:
                    token = 'x'
                out += token + ' | '
            print(out)
        print('-------------')


def get_all_states_impl(current_state, current_symbol, all_states):
    for i in range(0, BOARD_ROWS):
        for j in range(0, BOARD_COLS):
            if current_state.data[i][j] == 0:
                newState = current_state.next_state(i, j, current_symbol)
                newHash = newState.hash()
                if newHash not in all_states.keys():
                    isEnd = newState.is_end()
                    all_states[newHash] = (newState, isEnd)
                    if not isEnd:
                        get_all_states_impl(newState, -current_symbol, all_states)

def get_all_states():
    current_symbol = 1
    current_state = State()
    all_states = dict()
    all_states[current_state.hash()] = (current_state, current_state.is_end())
    get_all_states_impl(current_state, current_symbol, all_states)
    return all_states

# all possible board configurations
all_states = get_all_states()
print(len(all_states))
print(all_states[0])

5478
(<__main__.State object at 0x1118c7f98>, False)


In [104]:
current_state = State()
print(current_state)
print(current_state.hash())

0
